In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/')
dataset_path = '/content/drive/MyDrive/dataset-v1.1/'
import cv2
from google.colab.patches import cv2_imshow
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
!pip install ultralytics
import ultralytics
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

## Получение треков 
https://github.com/mikel-brostrom/yolov8_tracking, трекер ocsort.

Модель yolov8s, обучена на 100 эпохах (подробности можно уточнить у @apchoo11)

In [ ]:
!rm -r /content/yolov8_tracking/runs/track


In [ ]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/yolov8_tracking.git 
!cp yolov8_tracking/requirements.txt /content/
!pip install -r requirements.txt  
!mkdir /content/yolov8_tracking/weights

In [ ]:
!python3 yolov8_tracking/track.py --yolo-weights /content/yolov8m.pt --tracking-method ocsort --source /content/drive/MyDrive/dataset-v1.1/test/1.mp4 --save-txt --save-conf --save-crop --save-trajectories --classes 0 --save-vid 
!cp /content/yolov8_tracking/runs/track/exp/1.mp4 /content/drive/MyDrive/Helmet/tracks/Ocsort
!cp /content/yolov8_tracking/runs/track/exp/tracks/1.txt /content/drive/MyDrive/Helmet/tracks/Ocsort

In [ ]:
for i in range(2, 44):
    os.system(f'python3 yolov8_tracking/track.py --yolo-weights /content/drive/MyDrive/Helmet/models/best.pt --tracking-method ocsort --source /content/drive/MyDrive/dataset-v1.1/test/{i}.mp4 --save-txt --save-conf --save-crop --save-trajectories --classes 0 --save-vid ')
    os.system(f'cp /content/yolov8_tracking/runs/track/exp{i}/tracks/{i}.txt /content/drive/MyDrive/Helmet/tracks/Ocsort')
    os.system(f'cp /content/yolov8_tracking/runs/track/exp{i}/{i}.mp4 /content/drive/MyDrive/Helmet/tracks/Ocsort')

## Подсчёт входящих и исходящих (без учёта касок и жилетов)

In [ ]:
for N in range(1, 44):
    try:
        with open(f'/content/drive/MyDrive/Helmet/tracks/Ocsort/{N}.txt', 'r') as f:
            tracks = f.readlines()
        tracks = [t.strip().split() for t in tracks]
        tracks = [list(map(int, map(float, t)))[:-4] for t in tracks]           # Прошу не кидать тапками, это конвертация символов в цифры (str -> int).
        detections = True

    except:
        print(f'Video {N}: No detections')
        print()
        detections = False

    if detections:
        vidcap = cv2.VideoCapture(f'/content/drive/MyDrive/Helmet/tracks/Ocsort/{N}.mp4', )
        success, image = vidcap.read()

        gate_y = 375 * image.shape[0] // 640    # примерная координата турникета
        delta = 50 * image.shape[0] // 640      # эмпирическая поправка (она вроде не особо влияет, но пусть будет)
        gate_y -= delta
        res = image.shape[0]                    # размер видео по вертикали

        n_humans = 0                            # общее количество id в треке (далее может увеличиться)

        for t in tracks:
            n_humans = (max(t[1], n_humans))

        '''
        Предобработка треков.
        Иногда один и тот же id сначала проходит в одном направлении, а потом в другом, нужно посчитать его как одного входящего и одного выходящего.
        В данной реализации сравнивается первая и последняя координата объекта, поэтому если он сначала вошёл, а потом вышел, начальная и последняя координаты будут по одну сторону от турникета и алгоритм не сработает.
        Предпринята попытка форсированной смены id. Идея: после того как объект прошёл коридор - все дальнейшие id этого объекта заменяются на новые, и при последующем проходе, он будет считаться как новый объект с новым id.
        '''

        bb_y = [None] + [list() for _ in range(n_humans)]               # координата 'у' верхнего левого угла
        bb_h = [None] + [list() for _ in range(n_humans)]               # высота баундин бокса
        bb_frame = [None] + [list() for _ in range(n_humans)]           # номер кадра
        bb_start = [None] + [None for _ in range(n_humans)]             # с какой стороны заходит (= 0, если чел выходит,  = res, если заходит)

        for i, t in enumerate(tracks):
            id = t[1]
            frame_n = t[0]
            x, y, w, h = t[2:]
            bb_y[id].append(y)
            bb_h[id].append(h)
            bb_frame[id].append(frame_n)
            if i == 0:
                bb_start[id] = bb_y[id][0]
            else:
                if bb_y[id][-1] < bb_y[id][0]:
                    bb_start[id] = res
                else:
                    bb_start[id] = 0

            if len(bb_frame[id]) > 20:

                condition_in = bb_start[id] == 0 and (bb_y[id][-1] + bb_h[id][-1]) / res > 0.75 and bb_frame[id][-1] - bb_frame[id][-2] > 20          # условие смены id при проходе внутрь: 1) стартовая точка наверху, 2) нижняя граница бб пересекла порог (0,75 от высоты изображения) 3) бб не детектировался в течение 10 предыдущих кадров                                         # go in
                condition_out = bb_start[id] == res and bb_y[id][-1] / res < 0.35 and bb_frame[id][-1] - bb_frame[id][-2] > 20                        # условие смены id при проходе наружу: 1) стартовая точка внизу, 2) верхняя граница бб пересекла порог (0,35 от высоты изображения) 3) бб не детектировался в течение 10 предыдущих кадров

                if condition_in or condition_out:
                    # print(f'New human {id} -> {n_humans+1}, frame: {frame_n}')
                    # print()
                    n_humans += 1
                    bb_y.append([])
                    bb_h.append([])
                    bb_frame.append([])
                    bb_start.append(None)
                    for j in range(i+1, len(tracks)):       # во всех последующих детекциях данный id меняются новый
                        if tracks[j][1] == id:
                            tracks[j][1] = n_humans

        '''
        Собственно подсчёт.
        Если условная координата "y" турникета находится в интервале между начальной и конечной координатой объекта, то защитывается проход
        '''

        came_in = 0                                         # количество вошедших
        came_out = 0                                        # количество вышедших

        human_tracks = [None] + [list() for _ in range(n_humans)]               # нулевой элемент добавлен, чтобы нумерация начиналась с 1

        print(f'Track {N}):')

        for t in tracks:
            n = t[1]
            x, y, w, h = t[2:]
            box_center = x + w//2, y + h//2
            human_tracks[n].append(box_center[1])

        for i, h in enumerate(human_tracks):
            if h and h[0] < gate_y  < h[-1]:
                came_in += 1
                # print(f'          Human {i} came in')

            elif h and h[0] > gate_y > h[-1]:
                came_out += 1
                # print(f'          Human {i} came out')

        print()
        print(f'    {came_in} humans came in')
        print(f'    {came_out} humans came out')
        print()

Video 0: No detections

Track 1):

    3 humans came in
    0 humans came out

Track 2):

    3 humans came in
    0 humans came out

Track 3):

    3 humans came in
    0 humans came out

Track 4):

    4 humans came in
    0 humans came out

Track 5):

    0 humans came in
    0 humans came out

Track 6):

    1 humans came in
    4 humans came out

Track 7):

    0 humans came in
    4 humans came out

Track 8):

    0 humans came in
    5 humans came out

Track 9):

    1 humans came in
    1 humans came out

Track 10):

    0 humans came in
    2 humans came out

Track 11):

    0 humans came in
    3 humans came out

Track 12):

    0 humans came in
    4 humans came out

Track 13):

    2 humans came in
    4 humans came out

Track 14):

    0 humans came in
    4 humans came out

Track 15):

    2 humans came in
    0 humans came out

Track 16):

    0 humans came in
    4 humans came out

Track 17):

    0 humans came in
    6 humans came out

Track 18):

    4 humans came in


## Ошибки

Форсированная смена id, ocsort

In [ ]:
Видео 14: 0 вошли, 3 вышли // посчитано 0, 4 (один человек посчитан 2 раза, т.к. на нём детектится 2 бб)
Видео 30: 1 вошёл, 3 вышли // посчитано 1, 1 (плохо детектится первый выходящий, всем выходящим присваиваится id5)


COCO

In [ ]:
go_in_true = 3+3+3+4+1+1+2+2+4+1+1++1+2+1+1+1+1+7+2
go_out_true = 4+4+4+1+2+3+4+4+3+4+6+3+1+1+1+1+1+3+2+6+1+1
go_in_err = 0 
go_out_err = 1+2

err = (go_in_err + go_out_err)/(go_in_true + go_out_true)
1 - err

0.9702970297029703